In [1]:
import re
from IPython.display import clear_output
import time
import numpy as np


In [2]:
with open("input6.txt", "r") as file:
    data = file.read().splitlines()
    data_direc = [list(item) for item in data]
for i in range(len(data_direc)):
    for j in range(len(data_direc[i])):
        data_direc[i][j] = ""

In [3]:
def is_in_map(data):
    res = False
    for i in range(len(data)):
        if ("^" in data[i] or 
            ">" in data[i] or 
            "v" in data[i] or 
            "<" in data[i]):
            res = True
    return res

In [4]:
direction = {"^": (-1, 0),
             ">": (0, 1),
             "v": (1, 0),
             "<": (0, -1)}

In [5]:
turn = {"^": ">",
        ">": "v",
        "v": "<",
        "<": "^"}

In [6]:
data

['....#.....',
 '..#......#',
 '.........#',
 '..........',
 '.......#..',
 '..........',
 '....^.....',
 '........#.',
 '#.........',
 '......#...']

In [7]:
data_direc

[['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '']]

In [8]:
data_direc_temp = data_direc.copy()
data_direc_temp = np.asmatrix(data_direc)
data_direc_temp[2:, 2] = data_direc_temp[2:, 2] + "2"

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('<U1')) -> None

In [2262]:
data_direc_temp

matrix([['', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', ''],
        ['', '', '2', '', '', '', '', '', '', '']], dtype='<U1')

In [2263]:
np.asmatrix(data_direc)[5-1-1:5,1] + "^"

matrix([['^'],
        ['^']], dtype='<U2')

In [2264]:
def check_right(i, j, type, data, data_direc):
    if type == "<":
        above = np.hstack(np.asmatrix(data_direc)[:i, j][::-1]).tolist()[0]
        for ind, item in enumerate(above):
            if "^" in item and data[i][j] != "#":
                return True
            elif ">" in item and data[i-ind-1][j] == "#" and i-ind-1>=0:
                return True
            elif data[i][j] == "#":
                return False
            elif data[i-ind-1][j] == "#" and i-ind-1>=0:
                data_direc_temp = data_direc.copy()
                data_direc_temp = np.asmatrix(data_direc_temp)
                data_direc_temp[i-ind:i, j] = data_direc_temp[i-ind:i, j] + "^"
                print(data_direc_temp)
                return check_right(i-ind, j, "^", data, data_direc_temp)
        return False
    elif type == "^":
        right = np.asmatrix(data_direc)[i, j+1:].tolist()[0]
        for ind, item in enumerate(right):
            if ">" in item and data[i][j] != "#":
                return True
            elif "v" in item and data[i][j+ind+1] == "#":
                return True
            elif data[i][j] == "#":
                return False
            elif data[i][j+ind+1] == "#":
                data_direc_temp = data_direc.copy()
                data_direc_temp = np.asmatrix(data_direc_temp)
                data_direc_temp[i, j:j+ind] = data_direc_temp[i, j:j+ind] + ">"
                print(data_direc_temp)
                return check_right(i, j+ind, ">", data, data_direc_temp)
        return False
    elif type == ">":
        below = np.hstack(np.asmatrix(data_direc)[i+1:, j]).tolist()[0]
        for ind, item in enumerate(below):
            if "v" in item and data[i][j] != "#":
                return True
            elif "<" in item and data[i+ind+1][j] == "#":
                return True
            elif data[i][j] == "#":
                return False
            elif data[i+ind+1][j] == "#":
                data_direc_temp = data_direc.copy()
                data_direc_temp = np.asmatrix(data_direc_temp)
                data_direc_temp[i:i+ind, j] = data_direc_temp[i:i+ind, j] + "v"
                print(data_direc_temp)
                print(i,j)
                return check_right(i+ind, j, "v", data, data_direc_temp)
        return False
    elif type == "v":
        left = np.asmatrix(data_direc)[i, :j][::-1].tolist()[0]
        for ind, item in enumerate(left):
            if "<" in item and data[i][j] != "#":
                return True
            elif "^" in item and data[i][j-ind-1] == "#" and j-ind-1>=0:
                return True
            elif data[i][j] == "#":
                return False
            elif data[i][j-ind-1] == "#" and j-ind-1>=0:
                data_direc_temp = data_direc.copy()
                data_direc_temp = np.asmatrix(data_direc_temp)
                data_direc_temp[i, j-ind:j] = data_direc_temp[i, j-ind:j] + "<"
                #print(i,j-ind-1)
                print(data_direc_temp)
                return check_right(i, j-ind, "<", data, data_direc_temp)
        return False


In [2265]:
def replace_and_move(data, count, data_direc):
    for i in range(len(data)):
        regex = re.search("\^|\>|v|\<", data[i])
        try:
            if regex:
                pos = regex.start()
                type = regex.group()
                direc = direction[type]
                data[i] = data[i][:pos] + "X" + data[i][pos+1:]
                data_direc[i][pos] = data_direc[i][pos] + type
                #print(i, pos)
                if check_right(i, pos, type, data, data_direc):
                    count += 1
                    print(i, pos)
                if data[i+direc[0]][pos+direc[1]] == "#":
                    type = turn[type]
                    direc = direction[type]
                    data[i+direc[0]] = data[i+direc[0]][:pos+direc[1]] + type + data[i+direc[0]][pos+direc[1]+1:]
                    data_direc[i][pos] = data_direc[i][pos] + type
                elif pos + direc[1] < 0:
                    return (data, count, data_direc)              
                else:
                    data[i+direc[0]] = data[i+direc[0]][:pos+direc[1]] + type + data[i+direc[0]][pos+direc[1]+1:]
                return (data, count, data_direc)
        except:
            pass
    return (data, count, data_direc)

In [2266]:
count_pos = 0
while is_in_map(data):
    #clear_output(wait=True)
    #time.sleep(0.8)
    data, count_pos, data_direc = replace_and_move(data, count_pos, data_direc)
    #print(np.reshape(data, (10,1)))

[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '^' '>' '' '' '' '']
 ['' '' '' '' '^' '' '' '' '' '']
 ['' '' '' '' '^' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '^' '>' 'v' '' '' '']
 ['' '' '' '' '^' '' 'v' '' '' '']
 ['' '' '' '' '^' '' 'v' '' '' '']
 ['' '' '' '' '' '' 'v' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]
4 6
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '^' '>' 'v' '' '' '']
 ['' '' '' '' '^' '' 'v' '' '' '']
 ['' '' '' '' '^' '' 'v' '' '' '']
 ['' '' '' '' '' '' 'v' '' '' '']
 ['' '<' '<' '<' '<' '<' '' '' '' '']
 ['' '' '' '

In [2267]:
count_pos

7

In [2021]:
data

['.............',
 '.....#.#.....',
 '..#..XXXXX#..',
 '.....X...X.#.',
 '.....X...X...',
 '.....X...X...',
 '.#...X...X...',
 'XXXXXXXXXX#..',
 '.....X...#...',
 '.............']

In [ ]:
print(np.asmatrix(data_direc))

[['' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '^>' '>' '>' '>' '>v' '' '' '']
 ['' '' '' '' '' '^' '' '' '' 'v' '' '' '']
 ['' '' '' '' '' '^' '' '' '' 'v' '' '' '']
 ['' '' '' '' '' '^' '' '' '' 'v' '' '' '']
 ['<<' '<<' '<<' '<<' '<<' '<<' '<<' '<<' '<<' 'v<<' '<' '<' '<']
 ['' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '']]


In [ ]:
count = 0
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j] == "X":
            count += 1

In [1304]:
test = data.copy()

In [1162]:
count_pos

6

In [1174]:
count_pos

753

In [818]:
np.asmatrix(data_direc)

matrix([['', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '^', '>', '>', '>', '>', ''],
        ['', '', '', '', '^', '', '', '', 'v', ''],
        ['', '', '', '', '^', '', '', '', 'v', ''],
        ['', '', '^', '>', '^>', '>', '>', '', 'v', ''],
        ['', '', '^', '', '^', '', 'v', '', 'v', ''],
        ['', '', '<', '<', '^<', '<', '<v', '<', 'v', ''],
        ['', '^', '>', '>', '>', '>', 'v>', '>', '', ''],
        ['', '<', '<', '<', '<', '<', 'v', 'v', '', ''],
        ['', '', '', '', '', '', '', 'v', '', '']], dtype='<U2')

In [389]:
len(data[0])

130